In [1]:
import glob
import time
import uqer
import pickle
import datetime
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb
from uqer import DataAPI   #优矿api
from sklearn import linear_model
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

client = uqer.Client(token='18266a7c0ac9f8cdbe00f9b2ecb65f42316a5f78d9cc22ebabcbd923593356e4')
warnings.simplefilter(action='ignore', category=FutureWarning)

UQER SDK的DataAPI模块版本由84.0.72升级到84.0.82
127807@wmcloud.com 账号登录成功


## Useful Functions

In [2]:
def ticekrToStr(job_sum):
    job_sum = job_sum[job_sum.ticker<700000]
    job_sum.loc[job_sum.ticker<10,'temp']='00000'
    job_sum.loc[(job_sum.ticker<100)&(job_sum.ticker>=10),'temp']='0000'
    job_sum.loc[(job_sum.ticker<1000)&(job_sum.ticker>=100),'temp']='000'
    job_sum.loc[(job_sum.ticker<10000)&(job_sum.ticker>=1000),'temp']='00'
    job_sum.loc[job_sum.temp==job_sum.temp,'ticker'] = job_sum[job_sum.temp==job_sum.temp]['temp']+job_sum[job_sum.temp==job_sum.temp]['ticker'].astype(str)
    del job_sum['temp']
    job_sum['ticker'] = job_sum['ticker'].astype(str)
    return job_sum

def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

## Deal with raw data

In [3]:
file_location = '/data/liufengyuan/'

In [ ]:
####alpha####
path1 = file_location + 'raw_data/alphas_036'
filenames = glob.glob(path1 + '/*.csv')
result_list = []
for file in filenames:
    name = file[38:-4]
    print(name)
    alpha = pd.read_csv(file)
    alpha = alpha.rename(columns={"date": "tradeDate"})
    alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
    alpha['ticker'] = alpha['ticker'].astype(int)
    alpha = alpha.set_index(['ticker', 'tradeDate'])
    alpha = alpha.sort_index(ascending=True)
    result_list.append(alpha)
result = pd.concat(result_list, axis=1)
result.to_csv(file_location + 'pct5_cal/alphas_036.csv')
print(result)

In [ ]:
####alternative####
alter = pd.read_csv(file_location + 'raw_data/alternative_factors.csv')
alter = alter.iloc[:, 1:]
print(alter.shape)

In [ ]:
####combine alternative+alpha####
alpha = result.reset_index()
alter = alter.merge(alpha, on=['ticker', 'tradeDate'], how='left')
del result
del alpha
print(alter.shape)
alter = alter.dropna(thresh = 87)
alter = reduce_mem_usage(alter)
f_x_066 = alter.columns.values.tolist()[2:]
print(alter.shape)
print(f_x_066)
pickle.dump(f_x_066, open(file_location + "pct5_cal/f_x_036", "wb"))
alter.to_csv(file_location + 'pct5_cal/1alter_alphas_036.csv', index = False)

## Label

In [7]:
pd.read_csv(file_location + 'labels/PCT5.csv')

,Unnamed: 0,ticker,tradeDate,PCT5,PCT5_rank
0,0,1,2017-01-03,-0.001201,0.566613
1,1,1,2017-01-04,-0.001418,0.723032
2,2,1,2017-01-05,0.002189,0.869945
3,3,1,2017-01-06,-0.002079,0.870394
4,4,1,2017-01-09,-0.001967,0.873344
...,...,...,...,...,...
5017053,5046061,689009,2022-05-10,0.126496,0.968439
5017054,5046062,689009,2022-05-11,0.163312,0.974793
5017055,5046063,689009,2022-05-12,0.232190,0.989411
5017056,5046064,689009,2022-05-13,0.181071,0.976069


In [ ]:
file = 'label_vwap_pct1_full.csv'
name = file[6:-4]
print(name)
label = name + '_rank'
alpha = pd.read_csv('raw_data/' + file)
alpha

In [ ]:
alpha = alpha.loc[alpha['tradeDate'] >= '2017-01-03']
#alpha = alpha.rename(columns = lambda x: x[:-5] if (x != 'date') else 'tradeDate')
alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
alpha['ticker'] = alpha['ticker'].astype(int)
alpha = alpha.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
alpha[label] = alpha.groupby('tradeDate')[name].rank(pct = True)
alpha.to_csv('labels/{}.csv'.format(name), index=False)
print('----- Finished FILE {}------'.format(file))
alpha

## Uqer Data

In [ ]:
def get_target(ticker_name):
    # get close price and market value
    outData = DataAPI.MktEqudGet(ticker=ticker_name, beginDate='2017-07-25', endDate='2022-04-05', pandas="1")
    outData = outData[['ticker', 'tradeDate', 'closePrice', 'marketValue']]
    # get industry
    industry_data = DataAPI.EquIndustryGet(ticker=ticker_name, pandas="1")
    industry_name = industry_data.at[0, 'industryName1']
    # get listDate
    list_date = DataAPI.EquGet(ticker=ticker_name ,pandas="1")
    list_date = list_date.at[0, 'listDate']
    # combine data
    outData['industry'] = industry_name
    outData['listData'] = list_date
    outData = outData.sort_values(by=['tradeDate']).reset_index(drop=True)
    return outData[['ticker', 'tradeDate', 'marketValue', 'industry', 'listData']]

print('load data')
data = pd.read_csv(file_location + 'pct5_cal/1alter_alphas_036.csv', usecols=['ticker'])
print(data)
strData = ticekrToStr(data)
tickers = strData.ticker.unique()
tickers = tickers.tolist()

df_merge = get_target(tickers)
print('sort values')
df_merge = df_merge.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
print(df_merge)
df_merge.to_csv(file_location + 'pct1_cal/3uqer_data.csv', index = False)

df_merge.industry = df_merge.industry.fillna('其他')
industry_to_number = {}
for i, v in enumerate(df_merge.industry.sort_values().unique()):
    industry_to_number[v] = i+1
print(industry_to_number)
df_merge.industry = df_merge.industry.map(industry_to_number)
df_idst = pd.get_dummies(df_merge.industry, prefix='idst')
df = df_merge.merge(df_idst, left_index=True, right_index=True)
df['log_marketValue'] = df.marketValue.apply(np.log)
df.to_csv(file_location + 'pct5_cal/4uqer_idst_log.csv', index = False)

## Combine Data

In [ ]:
print('========LOADING AND COMBINING ALTER DATA========')
alpha = pd.read_csv('pct1_cal/alphas_066.csv')
print(alpha.shape)
print('========COMPLETE LOADING AND COMBINING ALTER DATA========')

print('========LOADING ALTER DATA========')
alter = pd.read_csv('raw_data/alternative_factors.csv')
alter = alter.iloc[:, 1:]
print(alter.shape)
print('========COMPLETE LOADING ALTER DATA========')

print('========COMBINING ALTER ALPHA DATA========')
alter = alter.merge(alpha, on=['ticker', 'tradeDate'], how='left')
del alpha
print(alter.shape)
print('========COMPLETE COMBINING ALTER ALPHA DATA========')

print('========REALLOCATE ALTER ALPHA DATA========')
alter = alter.dropna(thresh = 87)
alter = reduce_mem_usage(alter)
print(alter)
f_x_066 = alter.columns.values.tolist()[2:]
print(f_x_066)
pickle.dump(f_x_066, open("pct1_cal/f_x_066", "wb"))
print('store alphas 222')
alter.to_csv('pct1_cal/1alter_alphas_066.csv', index = False)
print('========COMPLETE STORING ALTER ALPHAS 066========')

#alter = pd.read_csv('pct1_cal/1alter_alphas_066.csv')

print('========LOADING UQER DATA========')
uqer_data = pd.read_csv('pct1_cal/4uqer_idst_log.csv')
print(uqer_data.shape)
print('========COMPLETE LOADING UQER DATA========')

print('========COMBINE ALTER ALPHA UQER DATA========')
alter = alter.merge(uqer_data, on=['ticker', 'tradeDate'], how='left')
alter = alter[~alter.log_marketValue.isnull()]
print(alter.shape)
del uqer_data
alter = reduce_mem_usage(alter)
print('========COMPLETE COMBINE ALTER ALPHA UQER DATA========')

print('get column lists')
f_index = ['ticker', 'tradeDate']
f_industry = pickle.load(open("pct1_cal/f_industry", "rb"))
f_x = pickle.load(open("pct1_cal/f_x_066", "rb"))
label_list = ['askbid_pct1_rank', 'openclose_pct1_rank']

print('========COMBINING LABEL DATA========')
path2 = 'labels'
filenames = glob.glob(path2 + '/*pct1.csv')
for file in filenames:
    print(file)
    label = pd.read_csv(file)
    label_name = label.columns.values.tolist()[-1]
    label = label[['ticker', 'tradeDate', label_name]]
    alter = alter.merge(label, on=['ticker', 'tradeDate'], how='left')
    del label
alter = reduce_mem_usage(alter)
print(alter.shape)
print('========COMPLETE COMBINING LABEL DATA========')

print('store raw data')
alter.to_csv('pct1_cal/alter_idst_alphas_066_labels_raw.csv', index = False)